In [ ]:
# Install missing dependencies
!pip install datasets scikit-learn transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from datasets import Dataset
from sklearn.model_selection import train_test_split
from google.colab import files
import pandas as pd

In [ ]:
# Load the trained model and tokenizer from Hugging Face Hub
model_name = "ccaug/modernbert-multiclass"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
# Upload the new dataset
print("Please upload the 'CICDDoSdataset.csv' file:")
uploaded = files.upload()

Please upload the 'CICDDoSdataset.csv' file:


Saving CICDDoSdataset.csv to CICDDoSdataset.csv


In [ ]:
# Load the dataset
df = pd.read_csv('CICDDoSdataset.csv', low_memory=False)

In [ ]:
# Remove leading/trailing spaces from column names (if needed)
df.columns = df.columns.str.strip()

In [ ]:
# Display unique labels
unique_labels = df['Label'].unique()
print(f"Number of unique labels: {len(unique_labels)}")
print(unique_labels)

Number of unique labels: 7
['NetBIOS' 'BENIGN' 'LDAP' 'Portmap' 'Syn' 'MSSQL' 'UDP']


In [ ]:
# Keep only the 7 desired labels
desired_labels = ['NetBIOS', 'BENIGN', 'LDAP', 'Portmap', 'Syn', 'MSSQL', 'UDP']
df = df[df['Label'].isin(desired_labels)]

In [ ]:
# Convert categorical labels to numerical labels
label_mapping = {label: i for i, label in enumerate(desired_labels)}
df['label'] = df['Label'].map(label_mapping)

In [ ]:
# Prepare text data (concatenate features as text input)
def convert_to_text(row):
    return " ".join(map(str, row))

df['text'] = df.drop(columns=['Label', 'label']).apply(convert_to_text, axis=1)

In [ ]:
# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df[['text', 'label']])

In [ ]:
# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

In [ ]:
# Split dataset into train and test sets ensuring balanced classes
train_data, test_data = train_test_split(df, test_size=0.5, stratify=df['label'], random_state=42, shuffle=True)

test_dataset = Dataset.from_pandas(test_data[['text', 'label']])
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [ ]:
# Predict function using the model directly
def predict_batch(batch):
    # Tokenize the input text
    inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)

    # Get predictions
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get predicted labels (argmax to choose the class with the highest score)
    predicted_labels = logits.argmax(dim=-1).cpu().numpy()

    return {"predictions": predicted_labels}

In [ ]:
# Apply prediction with smaller batch size
predictions = test_dataset.map(predict_batch, batched=True, batch_size=6)  # Reduce batch size here

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [ ]:
# Get the predicted labels and true labels
pred_labels = predictions['predictions']
true_labels = predictions['label']

In [ ]:
# Calculate metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')
f1 = f1_score(true_labels, pred_labels, average='weighted')
conf_matrix = confusion_matrix(true_labels, pred_labels)

In [ ]:
# Display results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=desired_labels))

print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9710
Precision: 0.9720
Recall: 0.9710
F1 Score: 0.9654

Classification Report:
              precision    recall  f1-score   support

     NetBIOS       1.00      0.99      0.99      1999
      BENIGN       0.98      0.99      0.99       909
        LDAP       0.93      0.99      0.96       997
     Portmap       1.00      0.05      0.10        95
         Syn       0.95      1.00      0.97      1000
       MSSQL       0.97      0.93      0.95      1000
         UDP       0.97      0.99      0.98      1000

    accuracy                           0.97      7000
   macro avg       0.97      0.85      0.85      7000
weighted avg       0.97      0.97      0.97      7000


Confusion Matrix:
[[1973    0    0    0    0    1   25]
 [   0  904    1    0    1    1    2]
 [   0    0  991    0    0    6    0]
 [   0   22    4    5   55    9    0]
 [   0    0    0    0 1000    0    0]
 [   0    0   64    0    0  932    4]
 [   0    0    0    0    0    8  992]]
